# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import create_tables

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
# import os 
# os.getcwd()


In [5]:
song_files = get_files('/home/workspace/data/song_data')
#print(song_files)

In [6]:
#song_files'
#type(song_files)
# import json 
# import glob 
# file = [] 
# # with open("result.json", "w") as outfile:
# #     for f in song_files:
# #         with open(f, "rb") as infile: 
# #             file_data = json.load(infile)
# #             file + = file_data
# #         json.dump(file, outfile)
# # for f in song_files:
# #     with open(f, "rb") as infile: 
# #         file.append(json.load(infile))
        
# with open("new_file2.json","wb") as outfile:
#     outfile.write('[{}]'.format(
#     ','.join([open(f,"rb").read() for f in song_files])))

In [7]:
filepath = '/home/workspace/data/song_data/A/B/B/TRABBLU128F93349CF.json'

In [8]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNNKDK1187B98BBD5,45.80726,Zagreb Croatia,15.9676,Jinx,407.37914,1,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
song_data = (df[['song_id','title','artist_id','year','duration']].astype(object).values)
#song_data.append(df['duration'].astype(float))
song_data = song_data.tolist()
song_data = song_data[0]
# song_data[4] = str(song_data[4])
# song_data[3] = str(song_data[3])

In [10]:
#song_tuple = ('SOFNOQK12AB01840FC','Kutt Free (DJ Volume Remix)', 'ARNNKDK1187B98BBD5', 0, 407.37914)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

In [13]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values
artist_data = artist_data.tolist()
artist_data = artist_data[0]
artist_data

['ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.80726, 15.967600000000001]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files('data/log_data')

In [16]:
#log_files

In [17]:
filepath = '/home/workspace/data/log_data/2018/11/2018-11-29-events.json'

In [18]:
df = pd.read_json(filepath, lines=True)
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df = df[df['page']=='NextSong']
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [20]:
t = pd.to_datetime(df['ts'])
t.head()
t.shape

(319,)

In [21]:
time_data = list((t.dt.time.tolist(),t.dt.hour.tolist(), t.dt.day.tolist(), t.dt.week.tolist(), t.dt.month.tolist(),\
             t.dt.year.tolist(), t.dt.weekday_name.tolist()))

column_labels = ['timestamp','hour', 'day', 'week', 'month', 'year', 'weekday']
#print((time_data))


In [22]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels,time_data)))
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,00:25:43.449657,0,1,1,1,1970,Thursday
1,00:25:43.449690,0,1,1,1,1970,Thursday
2,00:25:43.449841,0,1,1,1,1970,Thursday
3,00:25:43.449895,0,1,1,1,1970,Thursday
4,00:25:43.450033,0,1,1,1,1970,Thursday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [25]:
user_df = df[['userId','firstName','lastName','gender','level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [26]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [27]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [28]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    #song_select2 = ("""Select songs.song_id, artists.artist_id FROM ( songs JOIN artists ON songs.artist_id = artists.artist_id) where 
    #                songs.title = %s and artists.name = %s and songs.duration = %s """)
    #cur.execute(song_select2, (row.song, row.artist, row.length))
    #print(cur.execute(song_select2, (row.song, row.artist, row.length)))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
        #print(songid, artistid)
    else:
        songid, artistid = None, None
        #print("No values found")
        
    #songplay_data = pd.merge(df, results, left_on='artist', right_on='name')
    df['artistid'] = artistid
    df['songid'] = songid 
    #print(df.head())
    songplay_data = (df[['ts','userId', 'level','songid','artistid','sessionId','location','userAgent']].values[0].tolist())
    songplay_data = [str(i) for i in songplay_data]
    # songplay_id 	start_time 	user_id 	level 	artist_id 	session_id 	location 	user_agent
     # insert songplay record
#     songplay_data = list((str(row['itemInSession']), str(row['ts']), str(row['userId']), str(row['level']),\
#                           str(songid), str(artistid),\
#                           str(row['sessionId']), \
#                           str(row['location']),str(row['userAgent'])))
   # print(songplay_data)

    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.